In [4]:
%pip install rx

  Using cached Rx-3.2.0-py3-none-any.whl.metadata (4.6 kB)
Using cached Rx-3.2.0-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from rx import create

def generador(observer, schedule):
    observer.on_next("Hola")

    #Cuando se detecta el mensaje de erorr, ya no se continua con la ejecucion
    #observer.on_error("Error")
    observer.on_next("Mundo")

    #No lleva parametros; notificas y dices que la funcion ha terminado
    observer.on_completed()


observable = create(generador)

observable.subscribe(on_next = lambda s: print(f"Recibido: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error: {e}"),
                     on_completed = lambda: print("Finalizado"))

observable.subscribe(on_next = lambda s: print(f"Recibido2: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error2: {e}"),
                     on_completed = lambda: print("Finalizado2"))

Recibido: Hola
Recibido: Mundo
Recibido2: Hola
Recibido2: Mundo


In [13]:
from rx import of

observable = of('Hola', 'Adios', 'Mundo')

observable.subscribe(on_next = lambda s: print(f"Recibido: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error: {e}"),
                     on_completed = lambda: print("Finalizado"))


Recibido: Hola
Recibido: Adios
Recibido: Mundo
Finalizado


In [15]:
from rx import from_

observable = from_({1, 2, 3, 4, 5})

observable.subscribe(on_next = lambda s: print(f"Recibido: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error: {e}"),
                     on_completed = lambda: print("Finalizado"))

Recibido: 1
Recibido: 2
Recibido: 3
Recibido: 4
Recibido: 5
Finalizado


Podemos crear observers especializados heredando de la clase Observer; hacer que se escuche de teclado

In [17]:
from rx import create
from rx.core import Observer

#La clase printer, hereda de observer -----------------------------------------------------------------
class Printer(Observer):
    #Sobreescribimos la funcion de "on_next"
    def on_next(self, value):
        print(f"Recibido: {value}")

    def on_error(self, error):
        print(f"Mensaje de Error: {error}")

    def on_completed(self):
        print("Finalizado")

#FUNCION OBSERVADORA, LA QUE VA A MIRAR LO QUE SE PULSE ---------------------------------------------
def observer_teclado(observer, schedule):
    exit_ = False
    while not exit_:
        msg = input("Introduce un mensaje: ")
        if msg:
            if msg == "exit":
                exit_ = True
                observer.on_completed()
            else:
                observer.on_next(msg)
        else:
            observer.on_error("Error! No ha habido mensaje!")
            exit_ = True

observable = create(observer_teclado)

#para distinguirlos...
printer = Printer()

observable.subscribe(printer)

Recibido: hola
Finalizado
